In [27]:
import pandas as pd
import jpy_tools.parseSnake2 as jps

In [28]:
path_config = '/public1/software/liuzj/scripts/pipeline/online_hisat/config.yaml'
path_snakefile = '/public1/software/liuzj/scripts/pipeline/online_hisat/snakefile'

In [29]:
snf = jps.SnakeFile()

In [30]:
snh = jps.SnakeHeader(snf, path_config)
config = snh.getConfig()
snh

/public/home/liuzj/softwares/anaconda3/lib/python3.8/site-packages/jpy_tools/parseSnake2.py:93: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.yaml = yaml.load(open(self.path))


import pandas as pd
#configfile: "/public1/software/liuzj/scripts/pipeline/online_hisat/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"

In [31]:
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"

In [58]:
df_sra = pd.read_csv(config["sraRunTable"])[["Run", "Experiment"]]
df_hisat2mapping = (
    pd.DataFrame(df_sra.groupby("Experiment")["Run"].agg(list))
    .rename(columns={"Run": "srr"})
    .assign(srr=lambda df: df["srr"].map(lambda x: ",".join(x)))
)
df_hisat2mapping = df_hisat2mapping.assign(hisat2Index = config['hisat2Index'], resultDir = lambda df:df.index)
df_hisat2mapping.head()

,srr,hisat2Index,resultDir
Experiment,,,
ERX1492305,"ERR1421635,ERR1421719,ERR1421695,ERR1421656,ER...",/data/Zhaijx/liuzj/data/medtrA17_4/hisat2index...,ERX1492305
ERX1492306,"ERR1421696,ERR1421667,ERR1421744,ERR1421720,ER...",/data/Zhaijx/liuzj/data/medtrA17_4/hisat2index...,ERX1492306
ERX1492307,"ERR1421637,ERR1421673,ERR1421721,ERR1421746,ER...",/data/Zhaijx/liuzj/data/medtrA17_4/hisat2index...,ERX1492307
ERX1492308,"ERR1421674,ERR1421722,ERR1421747,ERR1421638,ER...",/data/Zhaijx/liuzj/data/medtrA17_4/hisat2index...,ERX1492308
ERX1492309,"ERR1421639,ERR1421748,ERR1421675,ERR1421724,ER...",/data/Zhaijx/liuzj/data/medtrA17_4/hisat2index...,ERX1492309


In [64]:
rule_hisat2mapping = jps.SnakeRule(snf, 'hisat2mapping', threads=2, wildCard='Experiment')
rule_hisat2mapping.addCode("""
df_sra = pd.read_csv(config["sraRunTable"])[["Run", "Experiment"]]
df_hisat2mapping = (
    pd.DataFrame(df_sra.groupby("Experiment")["Run"].agg(list))
    .rename(columns={"Run": "srr"})
    .assign(srr=lambda df: df["srr"].map(lambda x: ",".join(x)))
)
df_hisat2mapping = df_hisat2mapping.assign(hisat2Index = config['hisat2Index'], resultDir = lambda df:df.index)
""")
rule_hisat2mapping.addMetaDf('df_hisat2mapping', needAddRuleDirLs=['resultDir'], metaDf=df_hisat2mapping.head(2))
rule_hisat2mapping.addMain('params', ['srr', 'hisat2Index', 'resultDir'])
rule_hisat2mapping.setShell("""
mkdir -p {params.resultDir}
/public/apps/hisat2_2.1.0/bin/hisat2 -p {threads} -k 1 --dta -q --phred33 -x {params.hisat2Index} --sra-acc {params.srr} -S {params.resultDir}/unsorted.sam
samtools sort {params.resultDir}/unsorted.sam -@ {threads} -O bam -o {params.resultDir}/sorted.bam
samtools index {params.resultDir}/sorted.bam
rm {params.resultDir}/unsorted.sam
java -jar /public/apps/picard_2.20.2/picard.jar MarkDuplicates VALIDATION_STRINGENCY=SILENT REMOVE_DUPLICATES=true SORTING_COLLECTION_SIZE_RATIO=0.01 \
I={params.resultDir}/sorted.bam O={params.resultDir}/deduped.sorted.bam M={params.resultDir}/dedup.log
samtools index {params.resultDir}/deduped.sorted.bam
""")
rule_hisat2mapping

2021-11-04 11:12:52.134 | INFO     | jpy_tools.parseSnake2:addRule:55 - hisat2mapping step num: 1



## get parameter of rule `hisat2mapping` ##
df_sra = pd.read_csv(config["sraRunTable"])[["Run", "Experiment"]]
df_hisat2mapping = (
    pd.DataFrame(df_sra.groupby("Experiment")["Run"].agg(list))
    .rename(columns={"Run": "srr"})
    .assign(srr=lambda df: df["srr"].map(lambda x: ",".join(x)))
)
df_hisat2mapping = df_hisat2mapping.assign(hisat2Index = config['hisat2Index'], resultDir = lambda df:df.index)
for column in ['resultDir']:
    df_hisat2mapping[column] = resultDir + 'step1_hisat2mapping/' + df_hisat2mapping[column]
----------------
IN RULE
----------------
# parameter's dataframe of hisat2mapping: 
# | Experiment   | srr                                                    | hisat2Index                                                      | resultDir   |
# |:-------------|:-------------------------------------------------------|:-----------------------------------------------------------------|:------------|
# | ERX1492305   | ERR1421635,ERR1421719,ERR1421695,ERR1421656,ERR1

In [65]:
sna = jps.SnakeAll(snf, rule_hisat2mapping)
sna

rule all:
    input:
        hisat2mappingFinished = [resultDir + 'step1_hisat2mapping/' + "" + Experiment + ".finished" for Experiment in df_hisat2mapping.index],

In [66]:
snf.getMain(path_snakefile)

import pandas as pd
#configfile: "/public1/software/liuzj/scripts/pipeline/online_hisat/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"


## get parameter of rule `hisat2mapping` ##
df_sra = pd.read_csv(config["sraRunTable"])[["Run", "Experiment"]]
df_hisat2mapping = (
    pd.DataFrame(df_sra.groupby("Experiment")["Run"].agg(list))
    .rename(columns={"Run": "srr"})
    .assign(srr=lambda df: df["srr"].map(lambda x: ",".join(x)))
)
df_hisat2mapping = df_hisat2mapping.assign(hisat2Index = config['hisat2Index'], resultDir = lambda df:df.index)
for column in ['resultDir']:
    df_hisat2mapping[column] = resultDir + 'step1_hisat2mapping/' + df_hisat2mapping[column]

rule all:
    input:
        hisat2mappingFinished = [resultDir + 'step1_hisat2mapping/' + "" + Experiment + ".finished" for Experiment in df_hisat2mapping.index],

# parameter's dataframe of hisat2mapping: 
# | Experiment 